In [1]:
!pip install osmnx

                                              0.0/98.8 kB ? eta -:--:--
     ---------------------------------------- 98.8/98.8 kB ? eta 0:00:00
                                              0.0/1.1 MB ? eta -:--:--
     ------------------------------           0.8/1.1 MB 17.4 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 13.9 MB/s eta 0:00:00
                                              0.0/2.1 MB ? eta -:--:--
     --------------                           0.8/2.1 MB 24.1 MB/s eta 0:00:01
     ---------------------------------        1.7/2.1 MB 27.9 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 21.9 MB/s eta 0:00:00
                                              0.0/1.4 MB ? eta -:--:--
     --------------------------------         1.1/1.4 MB 23.3 MB/s eta 0:00:01
     ---------------------------------------- 1.4/1.4 MB 21.8 MB/s eta 0:00:00
                                              0.0/22.7 MB ? eta -:--:--
     -           


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import numpy as np
import pandas as pd
import folium

data=pd.read_json('https://data.cityofchicago.org/resource/t2qc-9pjd.json')
data.head()

,region,_region_id,_west,_east,_south,_north,_description,current_speed,_last_updt
0,North Park-Albany-Linconl Sq,3,-87.747456,-87.674590,41.960669,41.997946,Montrose to Devon. Cicero to Ravenswood,24.55,2023-07-18 00:20:48.0
1,Midway-Garfield Rdg-Clearing,17,-87.802188,-87.747456,41.764066,41.822792,71st to Pershing. Halem to Cicero,35.05,2023-07-18 00:20:49.0
2,South West Side,18,-87.747456,-87.683730,41.764066,41.822792,71st to Pershing. Cicero to Western,25.23,2023-07-18 00:20:49.0
3,Edge Water-Uptown,4,-87.674590,-87.646438,41.960669,41.997946,Montrose to Devon. Ravenswood to Lake Shore,26.59,2023-07-18 00:20:48.0
4,Lincoln Park-Lake View,8,-87.674590,-87.619112,41.910561,41.960669,North Ave to Montrose. Ravenswood to Lake Shore,28.64,2023-07-18 00:20:49.0


In [13]:
data['LONGITUDE']=(data['_east']+data['_west'])/2
data['LATITUDE']=(data['_north']+data['_south'])/2

In [14]:
data=data.drop(columns=['_east','_west','_south','_north','_description'],axis=1)

In [15]:
data.head()

,region,_region_id,current_speed,_last_updt,LONGITUDE,LATITUDE
0,North Park-Albany-Linconl Sq,3,24.55,2023-07-18 00:20:48.0,-87.711023,41.979308
1,Midway-Garfield Rdg-Clearing,17,35.05,2023-07-18 00:20:49.0,-87.774822,41.793429
2,South West Side,18,25.23,2023-07-18 00:20:49.0,-87.715593,41.793429
3,Edge Water-Uptown,4,26.59,2023-07-18 00:20:48.0,-87.660514,41.979308
4,Lincoln Park-Lake View,8,28.64,2023-07-18 00:20:49.0,-87.646851,41.935615


In [16]:
import ipywidgets as widgets
source=widgets.Dropdown(
    options=np.array(data['region']),
    description='Please select the source:',
)
source

Dropdown(description='Please select the source:', options=('North Park-Albany-Linconl Sq', 'Midway-Garfield Rd…

In [17]:
destination=widgets.Dropdown(
    options=data['region'],
    description='Please select the destination:',
)
destination

Dropdown(description='Please select the destination:', options=('North Park-Albany-Linconl Sq', 'Midway-Garfie…

In [18]:
s=source.value
d=destination.value

In [19]:

start=data[data['region']==s].iloc[:,:].values
end=data[data['region']==d].iloc[:,:].values

In [20]:
import osmnx as ox
import networkx as nx
ox.settings.log_console=True
ox.settings.use_cache=True

# define the start and end locations in latlng
start_latlng = start[0][5],start[0][4]
end_latlng = end[0][5],end[0][4]

# location where you want to find your route
place     = 'Chicago, Illinois, United States'

# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'

# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'

# create graph from OSM within the boundaries of some
# geocodable place(s)
graph = ox.graph_from_place(place, network_type = mode)

# find the nearest node to the start location
orig_node = ox.distance.nearest_nodes(graph, start_latlng[1],
                                      start_latlng[0])

# find the nearest node to the end location
dest_node = ox.distance.nearest_nodes(graph, end_latlng[1],
                                      end_latlng[0])
#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

In [21]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route,
                                          tiles='openstreetmap')
# Marker class only accepts coordinates in tuple form
start_latlng = (start_latlng[0],start_latlng[1])
end_latlng   = (end_latlng[0],end_latlng[1])
start_marker = folium.Marker(
            location = start_latlng,
            icon = folium.Icon(color='green'))
end_marker = folium.Marker(
            location = end_latlng,
            icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)


shortest_route_map

C:\Users\divya\AppData\Local\Temp\ipykernel_13852\2961681559.py:1: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph, shortest_route,


In [35]:
threeroutes=ox.distance.k_shortest_paths(graph,orig_node,dest_node,k=3)

In [36]:
for i in threeroutes:
  print(i)

[261128024, 261128026, 261262751, 5311567462, 305529057, 261147326, 261147308, 261158094, 261210190, 263905613, 261210188, 261127874, 261210187, 4081947035, 4081947037, 4081947039, 4081947041, 4081947043, 4081947046, 4081947049, 4081947050, 4081947052, 4081947054, 4081947055, 4081947058, 4081947061, 4081947063, 261181600, 289550366, 261199740, 261160361, 263322626, 261102104, 261105572, 261097154, 261210172, 5247967492, 261210170, 261145220, 261210169, 261174899, 261210168, 1445194506, 261141696, 261191248, 261193466, 261210167, 261147072, 315401875, 261210166, 261142599, 261210165, 4081947204, 261210164, 261210163, 261149702, 261210162, 261210161, 1445609232, 261209713, 1445609246, 261210159, 736035855, 526194637, 261123916, 261123915, 261123914, 261123912, 261123911, 261123909, 261123908, 261123907, 261123906, 261123903, 261123902, 261123901, 261123900, 261123899, 261123897, 261163661, 261163662, 261157761, 261163663, 261200659, 261198385, 261200658, 408447281, 261143108, 261137230, 

In [38]:
i=[261128024, 261128026, 261262751, 5311567462, 305529057, 261147326, 261147308, 261158094, 261210190, 263905613, 261210188, 261127874, 261210187, 4081947035, 4081947037, 4081947039, 4081947041, 4081947043, 4081947046, 4081947049, 4081947050, 4081947052, 4081947054, 4081947055, 4081947058, 4081947061, 4081947063, 261181600, 289550366, 261199740, 261160361, 263322626, 261102104, 261105572, 261097154, 261210172, 5247967492, 261210170, 261145220, 261210169, 261174899, 261210168, 1445194506, 261141696, 261191248, 261193466, 261210167, 261147072, 315401875, 261210166, 261142599, 261210165, 4081947204, 261210164, 261210163, 261149702, 261210162, 261210161, 1445609232, 261209713, 1445609246, 261210159, 736035855, 526194637, 261123916, 261123915, 261123914, 261123912, 261123911, 261123909, 261123908, 261123907, 261123906, 261123903, 261123902, 261123901, 261123900, 261123899, 261123897, 261163661, 261163662, 261157761, 261163663, 261200659, 261198385, 261200658, 408447281, 261143108, 261137230, 261137231, 261118575, 261118574]
#i=threeroutes[0]
j=[261128024, 261128026, 261262751, 5311567462, 305529057, 261147326, 261147308, 261158094, 261210190, 263905613, 261210188, 261127874, 261210187, 4081947035, 4081947037, 4081947039, 4081947041, 4081947043, 4081947046, 4081947049, 4081947050, 4081947052, 4081947054, 4081947055, 4081947058, 4081947061, 4081947063, 261181600, 289550366, 261199740, 261160361, 263322626, 261102104, 261105572, 261097154, 261210172, 5247967492, 261210170, 261145220, 261210169, 261174899, 261210168, 1445194506, 261141696, 261191248, 261193466, 261210167, 261147072, 315401875, 261210166, 261142599, 261210165, 4081947204, 261210164, 261210163, 261149702, 261210162, 261210161, 1445609232, 261209713, 1445609246, 261210159, 736035855, 526194637, 261123916, 261123915, 261123914, 261123912, 261123911, 261123909, 261123908, 261123907, 261123906, 261123903, 261123902, 261336487, 261310661, 261163661, 261163662, 261157761, 261163663, 261200659, 261198385, 261200658, 408447281, 261143108, 261137230, 261137231, 261118575, 261118574]
#j=threeroutes[1]
k=[261128024, 261128026, 261262751, 5311567462, 305529057, 261147326, 261147308, 261158094, 261210190, 263905613, 261210188, 261127874, 261210187, 4081947035, 4081947037, 4081947039, 4081947041, 4081947043, 4081947046, 4081947049, 4081947050, 4081947052, 4081947054, 4081947055, 4081947058, 4081947061, 4081947063, 261181600, 289550366, 261199740, 261160361, 263322626, 261102104, 261105572, 261097154, 261210172, 5247967492, 261210170, 261145220, 261210169, 261174899, 261210168, 1445194506, 261141696, 261191248, 261193466, 261210167, 261147072, 315401875, 261210166, 261142599, 261210165, 4081947204, 261210164, 261210163, 261149702, 261210162, 261210161, 1445609232, 261209713, 1445609246, 261210159, 736035855, 526194637, 261123916, 261123915, 261123914, 261123912, 261123911, 261123909, 261123908, 261123907, 261123906, 261123903, 261123902, 261123901, 261123900, 261310661, 261163661, 261163662, 261157761, 261163663, 261200659, 261198385, 261200658, 408447281, 261143108, 261137230, 261137231, 261118575, 261118574]
#k=threeroutes[2]

In [39]:
shortest_route_map = ox.plot_route_folium(graph,i,color='red',tiles='openstreetmap')
# Marker class only accepts coordinates in tuple form
start_latlng = (start_latlng[0],start_latlng[1])
end_latlng   = (end_latlng[0],end_latlng[1])
start_marker = folium.Marker(
                  location = start_latlng,
                  icon = folium.Icon(color='green'))
end_marker = folium.Marker(
                  location = end_latlng,
                  icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)

C:\Users\divya\AppData\Local\Temp\ipykernel_13852\489108840.py:1: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph,i,color='red',tiles='openstreetmap')


In [40]:
shortest_route_map

In [41]:

shortest_route_map = ox.plot_route_folium(graph,j, color='blue',tiles='openstreetmap')

# Marker class only accepts coordinates in tuple form
start_latlng = (start_latlng[0],start_latlng[1])
end_latlng   = (end_latlng[0],end_latlng[1])
start_marker = folium.Marker(
                  location = start_latlng,
                  icon = folium.Icon(color='green'))
end_marker = folium.Marker(
                  location = end_latlng,
                  icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)

C:\Users\divya\AppData\Local\Temp\ipykernel_13852\113467535.py:1: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph,j, color='blue',tiles='openstreetmap')


In [42]:
shortest_route_map

In [43]:

shortest_route_map = ox.plot_route_folium(graph,k, color='green',tiles='openstreetmap')

# Marker class only accepts coordinates in tuple form
start_latlng = (start_latlng[0],start_latlng[1])
end_latlng   = (end_latlng[0],end_latlng[1])
start_marker = folium.Marker(
                  location = start_latlng,
                  icon = folium.Icon(color='green'))
end_marker = folium.Marker(
                  location = end_latlng,
                  icon = folium.Icon(color='red'))
# add the circle marker to the map
start_marker.add_to(shortest_route_map)
end_marker.add_to(shortest_route_map)

C:\Users\divya\AppData\Local\Temp\ipykernel_13852\2542294773.py:1: UserWarning: The `folium` module has been deprecated and will be removed in a future release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  shortest_route_map = ox.plot_route_folium(graph,k, color='green',tiles='openstreetmap')


In [44]:
shortest_route_map

## DJIKSTRA ALGORITHM

In [ ]:
def func(u, v, d):
    node_u_wt = G.nodes[u].get("node_weight", 1)
    node_v_wt = G.nodes[v].get("node_weight", 1)
    edge_wt = d.get("weight", 1)
    return node_u_wt / 2 + node_v_wt / 2 + edge_wt

In [ ]:
G = nx.path_graph(5)
print(nx.dijkstra_path(G, 0, 4))

[0, 1, 2, 3, 4]


In [ ]:
G = nx.MultiDiGraph()
G.add_weighted_edges_from([(1, 2, 0.75), (1, 2, 0.5), (2, 3, 0.5), (1, 3, 1.5)])
nodes = nx.dijkstra_path(G, 1, 3)
edges = nx.utils.pairwise(nodes)
list((u, v, min(G[u][v], key=lambda k: G[u][v][k].get('weight', 1))) for u, v in edges)

[(1, 2, 1), (2, 3, 0)]

In [ ]:
shortest_route=nx.dijkstra_path(graph,orig_node,dest_node)
shortest_route

## A * ALGORITHM

In [ ]:
shortest_route=nx.astar_path(graph,orig_node,dest_node)
shortest_route